In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define parent directory
parent_dir = 'F:/thesis/data'

# Image dimensions
img_width, img_height = 224, 224
batch_size = 32

# Create data generators with augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Validation and testing data generators (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1. / 255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    os.path.join(parent_dir, 'train'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Load validation data
validation_generator = val_test_datagen.flow_from_directory(
    os.path.join(parent_dir, 'validation'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Load testing data
test_generator = val_test_datagen.flow_from_directory(
    os.path.join(parent_dir, 'test'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception

def build_xception_model(input_shape=(img_width, img_height, 3), num_classes=38):
    # Load Xception model with pretrained ImageNet weights
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model layers
    base_model.trainable = False

    # Add custom classification layers
    x = GlobalAveragePooling2D()(base_model.output)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

# Build Xception model
xception_model = build_xception_model()

# Display model summary
xception_model.summary()

In [ ]:
# Compile the model
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define number of epochs
epochs = 100

# Train the model
history = xception_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = xception_model.evaluate(test_generator)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")